In [ ]:
#初心者＆中級者向けの４つの書き方
#①tf.keras - Sequential API（初心者向け）
#②tf.keras - Sequential API
#③tf.keras - Functional API
#④tf.keras - Subclassing/Imperative API（おすすめらしい）(中級者向け)

In [1]:
%tensorflow_version 2.x

In [2]:
!pip install playground-data

  Created wheel for playground-data: filename=playground_data-1.1.1-py2.py3-none-any.whl size=20798 sha256=70297c953cbc92d06ebe1f7b76aaf1a26b5bc0731150b6f8613babcf648963c4
  Stored in directory: /root/.cache/pip/wheels/0a/83/26/9701478cd2f31df42fcc7d2cf0fa3fd6ff23cf8e44346166f3
Successfully built playground-data


In [3]:
!pip install tensorflow

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
import plygdata as pg
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyTwoGaussData

In [6]:
INPUT_FEATURES = 2
LAYER1_NEURONS = 3
LAYER2_NEURONS = 3
OUTPUT_RESULTS = 1

In [7]:
#①tf.keras - Sequential API（初心者向け）
model = tf.keras.models.Sequential([
                                    
#入力層&隠れ層１
layers.Dense(
    input_shape=(INPUT_FEATURES,),
    name='layer1',
    kernel_initializer='glorot_uniform',#重みの初期化の方法
    bias_initializer='zeros',#バイアスの初期化の方法
    units=LAYER1_NEURONS,#ユニットの数
    activation='tanh'),#活性化関数
#隠れ層２
layers.Dense(
    name='layer2',
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    units=LAYER2_NEURONS,
    activation='tanh'),
#出力層 
layers.Dense(
    name='layer_out',
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    units=OUTPUT_RESULTS,
    activation='tanh'),],
    name='sequential_constructor')

#modelの概要を見るにはmodel.summary()
model.summary()

Model: "sequential_constructor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 3)                 12        
                                                                 
 layer_out (Dense)           (None, 1)                 4         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [8]:
tf.keras.backend.clear_session()
del model

In [9]:
#②tf.keras - Sequential API
model = tf.keras.models.Sequential()

model.add(layers.Dense(
    input_shape=(INPUT_FEATURES,),
    name='layer1',
    units=LAYER1_NEURONS,
    activation='tanh'
))

model.add(layers.Dense(
    name='layer2',
    units=LAYER2_NEURONS,
    activation='tanh'
))

model.add(layers.Dense(
    units=OUTPUT_RESULTS,
    name='layer_out',
    activation='tanh'
))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 3)                 12        
                                                                 
 layer_out (Dense)           (None, 1)                 4         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [10]:
tf.keras.backend.clear_session()
del model

In [11]:
#③tf.keras - Functional API
activation1 = layers.Activation('tanh',
                                name='activation1')
activation2 = layers.Activation('tanh',
                                name='activation2')
acti_out = layers.Activation('tanh',
                                name='acti_out')
inputs = layers.Input(
    name='layer_in',
    shape=(INPUT_FEATURES,)
)

layer1 = layers.Dense(
    name='layer1',
    units=LAYER1_NEURONS
)

layer2 = layers.Dense(
    name='layer2',
    units=LAYER1_NEURONS
)

layer_out = layers.Dense(
    name='layer_out',
    units=OUTPUT_RESULTS
)

x1 = activation1(layer1(inputs))
x2 = activation2(layer2(x1))
outputs = acti_out(layer_out(x2))

model = tf.keras.Model(inputs=inputs, outputs=outputs,
                       name='model_constructor')

model.summary()


Model: "model_constructor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_in (InputLayer)       [(None, 2)]               0         
                                                                 
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 activation1 (Activation)    (None, 3)                 0         
                                                                 
 layer2 (Dense)              (None, 3)                 12        
                                                                 
 activation2 (Activation)    (None, 3)                 0         
                                                                 
 layer_out (Dense)           (None, 1)                 4         
                                                                 
 acti_out (Activation)       (None, 1)           

In [12]:
tf.keras.backend.clear_session()
del model

In [14]:
#④tf.keras - Subclassing/Imperative API（おすすめらしい）
activation1 = layers.Activation('tanh',
                               name='activation1')
activation2 = layers.Activation('tanh',
                                name='activation2')
acti_out = layers.Activation('tanh',
                             name='acti_out')

# tf.keras.Modelをサブクラス化してモデルを定義
class NeuralNetwork(tf.keras.Model):
  # ### レイヤーを定義 ###
  def __init__(self, *args, **kwargs):
    super(NeuralNetwork, self).__init__(*args, **kwargs)

    #入力層は定義不要、実際の入力によってきまる
    #隠れ層１
    self.layer1 = layers.Dense(
        name='layer1',
        units=LAYER1_NEURONS
    )
    
    #隠れ層２
    self.layer2 = layers.Dense(
        name='layer2',
        units=LAYER2_NEURONS
    )

    #出力層
    self.layer_out = layers.Dense(
        name='layer_out',
        units=OUTPUT_RESULTS
    )

  # ### フォワードパスを定義 ###
  def call(self, inputs, training=None):# 入力と、訓練／評価モード
    # 「出力＝活性化関数（第n層（入力））」の形式で記述
    x1 = activation1(self.layer1(inputs))
    x2 = activation2(self.layer2(x1))
    outputs = acti_out(self.layer_out(x2))
    return outputs

  # ### モデルの生成 ###
model = tf.keras.Model(inputs=inputs, outputs=outputs,
                         name='model_constructor')
model = NeuralNetwork()
  #この書き方だとmodel.summary()は使えない
  #なぜなら、Subclassingモデルは実行した段階で計算グラフが動的に構築される仕様だからである。
  #つまり、訓練を実行した後なら使える

In [15]:
import tensorflow.keras.backend as K

def tanh_accuracy(y_true, y_pred):           # y_trueは正解、y_predは予測（出力）
  threshold = K.cast(0.0, y_pred.dtype)              # -1か1かを分ける閾値を作成
  y_pred = K.cast(y_pred >= threshold, y_pred.dtype) # 閾値未満で0、以上で1に変換
  # 2倍して-1.0することで、0／1を-1.0／1.0にスケール変換して正解率を計算
  return K.mean(K.equal(y_true, y_pred * 2 - 1.0), axis=-1)

In [16]:
TRAINING_DATA_RATIO = 0.5
DATA_NOISE = 0.0

data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

X_train, y_train, X_valid, y_valid = pg.split_data(data_list, training_size=TRAINING_DATA_RATIO)

In [17]:
model.compile(tf.keras.optimizers.SGD(learning_rate=0.03), 'mean_squared_error',
              [tanh_accuracy])
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=15, epochs=100,
          verbose=1
)
model.predict([[0.1, -0.2]])

Epoch 1/100
17/17 [==============================] - 2s 28ms/step - loss: 0.3106 - tanh_accuracy: 0.9400 - val_loss: 0.0780 - val_tanh_accuracy: 1.0000
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 0.0532 - tanh_accuracy: 1.0000 - val_loss: 0.0370 - val_tanh_accuracy: 1.0000
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.0294 - tanh_accuracy: 1.0000 - val_loss: 0.0240 - val_tanh_accuracy: 1.0000
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 0.0203 - tanh_accuracy: 1.0000 - val_loss: 0.0176 - val_tanh_accuracy: 1.0000
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0154 - tanh_accuracy: 1.0000 - val_loss: 0.0139 - val_tanh_accuracy: 1.0000
Epoch 6/100
17/17 [==============================] - 0s 8ms/step - loss: 0.0124 - tanh_accuracy: 1.0000 - val_loss: 0.0115 - val_tanh_accuracy: 1.0000
Epoch 7/100
17/17 [==============================] - 0s 10ms/step - loss: 0.0104 - tanh_accur

array([[-0.08886333]], dtype=float32)

In [18]:
#訓練を実行した後なら見れる
model.summary()

Model: "neural_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              multiple                  9         
                                                                 
 layer2 (Dense)              multiple                  12        
                                                                 
 layer_out (Dense)           multiple                  4         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
